# 任务 3：使用 SageMaker Processing 和内置的 scikit-learn 容器执行数据处理

在此笔记本中，您将使用由 SageMaker Processing 提供和维护的 Docker 映像设置运行 scikit-learn 脚本所需的环境。

然后，您将使用 SageMaker Python SDK 中的 **SKLearnProcessor** 类来定义和运行 scikit-learn 处理作业。

最后，您将验证保存在 Amazon Simple Storage Service (Amazon S3) 中的数据处理结果。

**注意**：处理脚本会执行一些基本的数据处理，例如删除重复项、将目标列转换为包含两个标签的列、独热编码和以 80-20 分割率生成训练和测试数据集。

## 任务 3.1：环境设置

在此任务中，您将安装所需的程序包和依赖项。

您将设置 Amazon S3 存储桶来存储处理作业的输出，并让执行角色运行 SageMaker 处理作业。

In [ ]:
#install-dependencies
import logging
import boto3
import sagemaker
import pandas as pd
import numpy as np
from sagemaker.s3 import S3Downloader

sagemaker_logger = logging.getLogger("sagemaker")
sagemaker_logger.setLevel(logging.INFO)
sagemaker_logger.addHandler(logging.StreamHandler())

sagemaker_session = sagemaker.Session()

#Execution role to run the SageMaker Processing job
role = sagemaker.get_execution_role()
print("SageMaker Execution Role: ", role)

#S3 bucket to read the SKLearn processing script and writing processing job outputs
s3 = boto3.resource('s3')
for buckets in s3.buckets.all():
    if 'labdatabucket' in buckets.name:
        bucket = buckets.name
print("Bucket: ", bucket)

## 任务 3.2：运行 SageMaker 处理作业

在此任务中，您将导入并查看预处理数据集。

In [ ]:
#import-data
prefix = 'data/input'

S3Downloader.download(s3_uri=f"s3://{bucket}/{prefix}/adult_data.csv", local_path= 'data/')

shape=pd.read_csv("data/adult_data.csv", header=None)
shape.sample(5)

对于本实验，您将执行一些数据转换，例如删除重复项、将目标列转换为包含两个标签的列，以及对分类特征进行独热编码。

然后，您将使用 SKLearnProcessor 类来定义一个 scikit-learn 处理脚本并将其作为处理作业运行。有关此类的更多信息，请参阅 [SageMaker scikit-learn SKLearnProcessor](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/sagemaker.sklearn.html#sagemaker.sklearn.processing.SKLearnProcessor)。

要创建 SKLearnProcessor 类，请配置以下参数：
- **base_job_name**：处理作业名称的前缀
- **framework_version**：scikit-learn 版本
- **role**：SageMaker 执行角色
- **instance_count**：要运行处理作业的实例的数量
- **instance_type**：用于处理作业的 Amazon Elastic Compute Cloud (Amazon EC2) 实例的类型

In [ ]:
#scikit-learn-processor
import os
from sagemaker.sklearn.processing import SKLearnProcessor

# create a SKLearnProcessor
sklearn_processor = SKLearnProcessor(
    base_job_name="sklearn-preprocessor",
    framework_version="0.20.0", 
    role=role, 
    instance_count=1,
    instance_type="ml.m5.xlarge"
)


接着，使用 **SKLearnProcessor.run()** 方法将 **sklearn_preprocessing.py** 脚本作为处理作业运行。

要运行处理作业，请配置以下参数：
- **code**：预处理脚本的路径 
- **inputs and outputs**：预处理脚本的输入和输出路径（Amazon S3 输入和输出位置）
- **arguments**：预处理脚本的命令行参数（如训练和测试分割率）

完成处理作业大约需要 4–5 分钟。在作业运行时，您可以通过从文件浏览器打开 **sklearn_preprocessing.py** 文件来查看预处理脚本（已作为本实验的一部分进行预先配置）的源代码。

In [ ]:
#processing-job
from sagemaker.processing import ProcessingInput, ProcessingOutput

# Amazon S3 path prefix
input_raw_data_prefix = "data/input"
output_preprocessed_data_prefix = "data/output"
scripts_prefix = "scripts/smstudiofiles"
logs_prefix = "logs"

# Run the processing job
sklearn_processor.run(
    code="s3://" + os.path.join(bucket, scripts_prefix, "sklearn_preprocessing.py"),
    inputs=[ProcessingInput(source="s3://" + os.path.join(bucket, input_raw_data_prefix, "adult_data.csv"),
                            destination="/opt/ml/processing/input")],
    outputs=[
        ProcessingOutput(output_name="train_data", 
                         source="/opt/ml/processing/train",
                         destination="s3://" + os.path.join(bucket, output_preprocessed_data_prefix, "train")),
        ProcessingOutput(output_name="test_data", 
                         source="/opt/ml/processing/test",
                         destination="s3://" + os.path.join(bucket, output_preprocessed_data_prefix, "test")),
    ],
    arguments=["--train-test-split-ratio", "0.2"],
)

print("SKLearn Processing Job Completed.")

## 任务 3.3：验证数据处理结果

在此任务中，您将通过查看训练和测试输出数据集的前五行，验证您运行的处理作业的输出。

In [ ]:
#view-train-dataset
print("Top 5 rows from s3://{}/{}/train/".format(bucket, output_preprocessed_data_prefix))
!aws s3 cp --quiet s3://$bucket/$output_preprocessed_data_prefix/train/train_features.csv - | head -n5

In [ ]:
#view-validation-dataset
print("Top 5 rows from s3://{}/{}/validation/".format(bucket, output_preprocessed_data_prefix))
!aws s3 cp --quiet s3://$bucket/$output_preprocessed_data_prefix/test/test_features.csv - | head -n5

### 总结

恭喜！ 您已借助 SageMaker Processing 成功使用 SageMaker Python SDK 创建了 scikit-learn 处理作业并运行了该处理作业。

本实验的下一项任务侧重于使用 SageMaker Processing 和您自己的处理容器执行数据处理。

### 清理

您已完成此笔记本。要进入本实验的下一部分，请执行以下操作：

- 关闭此笔记本文件。
- 返回至实验会话，继续执行**任务 4：使用您自己的容器执行数据处理**。